# NLP Project
## Lab 02
Nelson VICEL--FARAH\
Karen KASPAR\
Romain BRAND

## The datasets

In [1]:
from datasets import load_dataset

In [2]:
imdb_dataset = load_dataset("imdb")

Reusing dataset imdb (/home/miolith/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### 1. How many splits does the dataset has? (1 point)
#### On observe que le dataset IMDb a 3 splits différents
### 2. How big are these splits? (1 point)
#### Le **train** split a 25 000 lignes, **test** 25 000 lignes et **unsupervised** avec 50 000 lignes

### 3. What is the proportion of each class on the supervised splits? (1 point)

In [4]:
import pandas as pd

train_df = pd.DataFrame(imdb_dataset["train"])

train_df["label"].value_counts()

0    12500
1    12500
Name: label, dtype: int64

In [5]:
test_df = pd.DataFrame(imdb_dataset["test"])

test_df["label"].value_counts()

0    12500
1    12500
Name: label, dtype: int64

#### Il y a 50% de chaque classe dans chaque split supervisé. Le dataset est équilibré.

## Naive Bayes classifier

### 1. (2 points) Take a look at the data and create an adapted preprocessing function.

On a fait un transformer sklearn personnalisé pour pouvoir l'ajouter directement dans la pipeline

In [6]:
from string import punctuation
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        return self
    
    def transform(self, text_list, label = None):
        new_text_list = text_list.copy()
        # i. Lower case the text
        new_text_list = new_text_list.str.lower()
        
        # ii. Remove punctuation
        new_text_list = new_text_list.str.replace('[{}]'.format(punctuation), ' ', regex=True)
        
        return new_text_list

### 2. (4 points) Implement and train a naive Bayes classifier on the training data.

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(CustomTransformer(), CountVectorizer(), MultinomialNB())

pipe.fit(train_df["text"], train_df["label"])

Pipeline(steps=[('customtransformer', CustomTransformer()),
                ('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

### 3. (1 point) Report the accuracy on both training and test set

In [8]:
train_accuracy = pipe.score(train_df["text"], train_df["label"])
print('The accuracy on the train set is', train_accuracy)

The accuracy on the train set is 0.89808


In [9]:
test_accuracy = pipe.score(test_df["text"], test_df["label"])
print('The accuracy on the test set is', test_accuracy)

The accuracy on the test set is 0.8136


### 4. (1 point) Why is accuracy a sufficient measure of evaluation here?
#### La précision est suffisante dans notre cas car le dataset est équilibré, donc on peut facilement juger qu'une précision supérieure à 50% est meilleure qu'un algorithme aléatoire.

### 6. (1 point) Take at least 2 wrongly classified example from the test set and try explaining why the model failed.

In [10]:
test_df["prediction"] = pipe.predict(test_df["text"])
test_df["prediction"]

0        0
1        0
2        0
3        0
4        0
        ..
24995    0
24996    0
24997    1
24998    0
24999    0
Name: prediction, Length: 25000, dtype: int64

In [11]:
failed_sample1 = test_df[(test_df["prediction"] == 1) & (test_df["label"] == 0)].iloc[1,:]["text"]
failed_sample2 = test_df[(test_df["prediction"] == 0) & (test_df["label"] == 1)].iloc[1,:]["text"]

#### Cet exemple est un faux positif. L'auteur est vraisemblablement ironique, ce qui aurait pu tromper l'algorithme de NaiveBayes.

In [12]:
failed_sample1

"Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an assistant to a foul-mouthed, eccentric, once-famous and now-forgotten actress Evie Walton, (Julie Walters), that he finally finds himself in true 'Harold and Maude' fashion. Of course, Evie is deeply unhappy herself and it's only when these two sad sacks find each other that they can put their mutual misery aside and hit the road to happiness.<br /><br />Of course it's corny and sentimental and very predictable but it has a hard side to it, too and Walters, who could sleep-walk her way through this sort of thing if she wanted, is excellent. It's when she puts the craziness to one side and finds the pathos in the character, (like hitting the bottle and throwing up in the sink), that she's at her 

#### Cet exemple est un faux négatif. Ce commentaire est une description de l'oeuvre plutôt qu'une opinion explicite, les adjectifs qualificatifs sont rarement utilisés pour expliciter si le film a été apprécié ou non.

In [13]:
failed_sample2

'This was a bold movie to hit Indian cinemas when it was released. The first movie to perhaps openly depict lesbian tendencies amongst Indian women. The leading actress of Indian cinema Shabana Azmi added substance to the movie with her hot passionate scenes with Nandita Das.<br /><br />The movie oozed with sexuality and the director used sex in the best way possible. The sex was not for erotic purposes but was in the context of the movie. The scene where Nandita Das loses her virginity to her husband certainly was the first of its kind in Indian cinemas.<br /><br />Good acting by all the actors especially Nandita Das amidst criticism from the Indian public'

![alt text](https://media.discordapp.net/attachments/465505646607859714/1026859583853051985/memenlp.png?width=311&height=400 "Title")

# Stemming

### 1. (2 points) Add stemming or lemmatization to your pretreatment.

In [11]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/miolith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import re

re_word = re.compile(r"^\w+$")
def stemming(text):
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in word_tokenize(text) if re_word.match(word)]
    return " ".join(stemmed)

class StemmingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        return self
    
    def transform(self, text_list, label = None):
        new_text_list = text_list.copy()
        
        return new_text_list.apply(stemming)

### 2. (1 point) Train and evaluate your model again with these pretreatment.

In [13]:
pipe_stem = make_pipeline(CustomTransformer(), StemmingTransformer(), CountVectorizer(), MultinomialNB())

pipe_stem.fit(train_df["text"], train_df["label"])

Pipeline(steps=[('customtransformer', CustomTransformer()),
                ('stemmingtransformer', StemmingTransformer()),
                ('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

### 3. (1 point) Are the results better or worse? Try explaining why the accuracy changed.

In [16]:
train_accuracy = pipe_stem.score(train_df["text"], train_df["label"])
print('The accuracy on the train set is', train_accuracy)

The accuracy on the train set is 0.88404


In [17]:
test_accuracy = pipe_stem.score(test_df["text"], test_df["label"])
print('The accuracy on the train set is', test_accuracy)

The accuracy on the train set is 0.80616


Avec le stemming, la précision a diminué de 1% du fait que cette méthode implique potentiellement de la perte d'information.